# 15. AMES HOUSING: FEATURE ENGINEERING
---

The goal of this chapter is to:
- deal with missing values
- create new features
- scale numerical features
- encode categorical features
- create a preprocessing pipeline ready for model testing

## 1. Introducing the Data

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 99)
pd.set_option("display.max_rows", 999)
pd.set_option('precision', 3)

ames = pd.read_csv('data/Ames_Housing1_train')
print(ames.shape)
ames.head()

(2344, 80)


,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,20,RL,80.0,10400.0,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,7,5,1976,1976,Gable,CompShg,HdBoard,HdBoard,BrkFace,189.0,TA,TA,CBlock,Gd,TA,No,Unf,0.0,Unf,0.0,1090.0,1090.0,GasA,TA,Y,SBrkr,1370.0,0.0,0.0,1370.0,0.0,0.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,479.0,TA,TA,Y,0.0,0.0,0.0,0.0,0.0,0.0,NaN,MnPrv,NaN,0.0,6,2009,WD,Family,152000.0
1,60,RL,NaN,28698.0,Pave,NaN,IR2,Low,AllPub,CulDSac,Sev,ClearCr,Norm,Norm,1Fam,2Story,5,5,1967,1967,Flat,Tar&Grv,Plywood,Plywood,None,0.0,TA,TA,PConc,TA,Gd,Gd,LwQ,249.0,ALQ,764.0,0.0,1013.0,GasA,TA,Y,SBrkr,1160.0,966.0,0.0,2126.0,0.0,1.0,2,1,3,1,TA,7,Min2,0,NaN,Attchd,1967.0,Fin,2.0,538.0,TA,TA,Y,486.0,0.0,0.0,0.0,225.0,0.0,NaN,NaN,NaN,0.0,6,2009,WD,Abnorml,185000.0
2,90,RL,70.0,9842.0,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,NAmes,Norm,Norm,Duplex,1Story,4,5,1962,1962,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,GasA,TA,Y,SBrkr,1224.0,0.0,0.0,1224.0,0.0,0.0,2,0,2,2,TA,6,Typ,0,NaN,CarPort,1962.0,Unf,2.0,462.0,TA,TA,Y,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,3,2007,WD,Normal,101800.0
3,90,RL,60.0,7200.0,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,Duplex,1Story,4,5,1949,1950,Gable,CompShg,BrkFace,Stone,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,Wall,Fa,N,FuseF,1040.0,0.0,0.0,1040.0,0.0,0.0,2,0,2,2,TA,6,Typ,0,NaN,Detchd,1956.0,Unf,2.0,420.0,TA,TA,Y,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,6,2009,WD,Normal,90000.0
4,190,RM,63.0,7627.0,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Artery,Norm,2fmCon,2Story,4,6,1920,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,Fa,TA,BrkTil,Fa,Po,No,Unf,0.0,Unf,0.0,600.0,600.0,GasA,Gd,N,SBrkr,1101.0,600.0,0.0,1701.0,0.0,0.0,2,0,4,2,Fa,8,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,N,0.0,0.0,148.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,10,2009,WD,Normal,94550.0


## 2. Dealing with Continuous Missing Values

In [3]:
continuous = ['Lot Frontage', 'Lot Area', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2',
              'Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Gr Liv Area',
              'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Low Qual Fin SF', 'Enclosed Porch', 
              '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val', 'SalePrice']
cont_df = ames[continuous].copy()
cont_null_counts = cont_df.isnull().sum()
cont_null = cont_df[cont_null_counts[cont_null_counts!=0].index]
cont_null = cont_null.isnull().sum().sort_values()
cont_null

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Area        1
Mas Vnr Area      19
Lot Frontage     393
dtype: int64

In [4]:
print(ames['Lot Frontage'].mean())
ames.groupby(['MS SubClass'])['Lot Frontage'].mean()

69.2075858534085


MS SubClass
20     77.476
30     60.874
40     53.000
45     57.286
50     63.883
60     78.693
70     64.010
75     75.789
80     78.657
85     72.296
90     71.425
120    44.248
150       NaN
160    27.556
180    27.000
190    68.935
Name: Lot Frontage, dtype: float64

We want to group by `MS SubClass` and then fill in the mean of the group into missing values. If the group mean doesn't exist, we will fill in the mean of the entire column!

In [9]:
cont_miss_cols = ['BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF',
                 'Total Bsmt SF', 'Garage Area', 'Mas Vnr Area']
for col in cont_miss_cols:
    cont_df[col] = cont_df[col].fillna(mean)
    
cont_df[cont_miss_cols].isnull().sum().sum()

0

In [11]:
cont_df['Lot Frontage'] = ames.groupby(['MS SubClass'])['Lot Frontage'].apply(
lambda x:x.fillna(x.mean()))
cont_df['Lot Frontage'] = cont_df['Lot Frontage'].fillna(mean)
cont_df['Lot Frontage'].isnull().sum()

0

## 3. Continuous Missing Values Transformers

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin

class ContMissFiller(BaseEstimator, TransformerMixin):    
    def __init__(self):
        self    
        
    def fit(self, df, y = None):               
        return self

    def transform(self, df):        
        df = df.copy()
        miss_cols = ['BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF',
                 'Total Bsmt SF', 'Garage Area', 'Mas Vnr Area']
        for col in miss_cols:
            df[col] = df[col].fillna(mean)        
        return df
    
class LotFrontFiller(BaseEstimator, TransformerMixin):    
    def __init__(self):
        self    
        
    def fit(self, df, y = None):               
        return self

    def transform(self, df):        
        use_cols = ['MS SubClass', 'Lot Frontage']
        df = df.copy()
        df['Lot Frontage'] = df.groupby(['MS SubClass'])['Lot Frontage'].apply(
            lambda x:x.fillna(x.mean()))
        df['Lot Frontage'] = df['Lot Frontage'].fillna(mean)
        return df

In [19]:
# pipeline and testing
from sklearn.pipeline import Pipeline

test_pipe = Pipeline(steps = [
    ('filler1', ContMissFiller()),
    ('filler2', LotFrontFiller())
])
test_df = ames[continuous + ['MS SubClass']]
test_df_ = test_pipe.fit_transform(test_df)
test_df_.isnull().sum()

Lot Frontage       0
Lot Area           0
Mas Vnr Area       0
BsmtFin SF 1       0
BsmtFin SF 2       0
Bsmt Unf SF        0
Total Bsmt SF      0
1st Flr SF         0
2nd Flr SF         0
Gr Liv Area        0
Garage Area        0
Wood Deck SF       0
Open Porch SF      0
Low Qual Fin SF    0
Enclosed Porch     0
3Ssn Porch         0
Screen Porch       0
Pool Area          0
Misc Val           0
SalePrice          0
MS SubClass        0
dtype: int64

## 4. Dealing with Discrete Missing Values

In [20]:
discrete = ['Year Built', 'Year Remod/Add', 'Garage Yr Blt', 'Garage Cars', 'Bsmt Full Bath',
           'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr',
           'TotRms AbvGrd', 'Fireplaces', 'Mo Sold', 'Yr Sold']
disc_df = ames[discrete].copy()
disc_null_counts = disc_df.isnull().sum()
disc_null = disc_df[disc_null_counts[disc_null_counts!=0].index]
disc_null = disc_null.isnull().sum().sort_values()
disc_null

Garage Cars         1
Bsmt Full Bath      1
Bsmt Half Bath      1
Garage Yr Blt     122
dtype: int64

In [30]:
ames.groupby(['MS SubClass'])['Garage Yr Blt'].median().astype(int)

MS SubClass
20     1975
30     1938
40     1949
45     1950
50     1950
60     1999
70     1932
75     1941
80     1976
85     1977
90     1976
120    2003
150    1980
160    1982
180    1989
190    1960
Name: Garage Yr Blt, dtype: int32

In [31]:
ames.groupby(['MS SubClass'])['Garage Yr Blt'].median()

MS SubClass
20     1975.0
30     1938.5
40     1949.0
45     1950.0
50     1950.0
60     1999.0
70     1932.5
75     1941.0
80     1976.0
85     1977.0
90     1976.0
120    2003.0
150    1980.0
160    1982.0
180    1989.0
190    1960.0
Name: Garage Yr Blt, dtype: float64

In [25]:
ames.groupby(['MS SubClass'])['Garage Yr Blt'].mean()

MS SubClass
20     1979.331
30     1945.848
40     1952.400
45     1951.273
50     1952.586
60     1994.613
70     1944.202
75     1952.737
80     1977.628
85     1978.564
90     1970.850
120    1997.960
150    1980.000
160    1987.500
180    1988.250
190    1955.735
Name: Garage Yr Blt, dtype: float64

In [29]:
ames.groupby(['MS SubClass'])['Garage Yr Blt'].value_counts(dropna=False)

MS SubClass  Garage Yr Blt
20           2007.0           51
             2006.0           43
             2005.0           38
             1956.0           32
             2008.0           28
             1968.0           27
             1959.0           24
             1954.0           23
             1960.0           23
             NaN              22
             1977.0           22
             1957.0           21
             1958.0           20
             1966.0           20
             2004.0           20
             1974.0           19
             1961.0           18
             1963.0           17
             1955.0           16
             1967.0           16
             2001.0           16
             2002.0           16
             2003.0           16
             1962.0           15
             1953.0           14
             1964.0           14
             1965.0           13
             1950.0           12
             1952.0           12
             199